In [1]:
import pandas as pd

In [2]:
xls = pd.ExcelFile(r"data/‏‏raw.xlsx")
groups_xls = pd.ExcelFile(r"data/טבלת הקצאה רנדומלית לתנאי הניסוי והבקרה.xlsx")

### Transform sheets to column

#### columns

In [3]:
aliases =  {
    'raw_id': [';', 'קוד', 'קוד מטופל', 'קוד נבדק'],
    'clinic_application_date': ['תאריך פנייה למרפאה'],
    'intake_date': ['תאריך אינטייק'],
    'group': ['הקצאה רנדומלית', 'סוג טיפול'],
    'signing_date': ['תאריך חתימה'],
    'therapy_start_date': ['תאריך תחילת התערבות'],
    'therapy_end_date': ['שאלוני סוף התערבות (8 שבועות)', 'תאריך סיום התערבות'],
    'suiteable_for_pp': ['מתאים לתומך ע"י קלינאי 1. כן 0. לא', 'מתאים לתומך ע"י קלינאי 1. כן 0. לא', 'מתאימים לתומך? 1.כן 0.לא', 'מתאימים לתומך? 1.כן 0.לא'],
    "First name": ['שם פרטי', 'שם פרטי של המטופל'],
    "Last name": ['שם משפחה', 'שם משפחה של המטופל']
}

rename_map = {col: key for key, value in aliases.items() for col in value}
columns = list(aliases.keys()) + ['sheet', 'clean_id']

In [4]:
def drop_s(x):
    if pd.isna(x):
        return x
    elif x.endswith('s'):
        return x[:-1]
    else: 
        return x


In [5]:
data_sets = []

for sheet in xls.sheet_names:
    sheet_df = xls.parse(sheet)
    sheet_df['sheet'] = sheet
    sheet_df.rename(columns=rename_map, inplace=True, errors='ignore')
    
    sheet_df["clean_id"] = sheet_df["raw_id"].astype(str).str.lower()
    sheet_df["clean_id"] = sheet_df["clean_id"].apply(drop_s)
    if "group" in sheet_df.columns:
        sheet_df["group"] = sheet_df["group"].replace({
            'Stepped care': "Stepped Care"})

    sheet_columns = [col for col in columns if col in sheet_df.columns]
    sheet_df = sheet_df[sheet_columns]
    data_sets.append(sheet_df)


df = pd.concat(data_sets)

In [6]:
groups_data_sets = []
for sheet in groups_xls.sheet_names:
    groups_df = groups_xls.parse(sheet)

    groups_df = groups_df.dropna(subset=["Participant code"])
    groups_df["clean_id"] = groups_df["Participant code"].astype(str).str.lower()
    groups_df["clean_id"] = groups_df["clean_id"].apply(drop_s)

    groups_df = groups_df.rename({'Assignment': 'group',
                                 "Participant code": "raw_id"}, axis=1)
    
    groups_df["group"] = groups_df["group"].replace({
        "ARM 1= Stepped care": "Stepped Care",
        "ARM 2=CAU": "CAU",
        'ARM 2= CAU': "CAU"
    })

    groups_data_sets.append(groups_df[['clean_id', 'raw_id', 'group']])


groups_df_concat = pd.concat(groups_data_sets)

In [7]:
mapping_groups = {}
for _, raw in groups_df_concat.iterrows():
    
    mapping_groups[raw.clean_id] = raw.group

df["group"] = df.group.fillna(df.clean_id.map(mapping_groups))

### ID processing

In [8]:
df.to_csv("data\preprocessed.csv", index=0)

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\USER\AppData\Local\Temp\ipykernel_5168\4067398792.py:1: SyntaxWarning: invalid escape sequence '\p'
  df.to_csv("data\preprocessed.csv", index=0)
